In [42]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [34]:
browser = webdriver.Chrome("./chromedriver")
browser.get("https://www.linkedin.com")

In [35]:
username = browser.find_element_by_id("session_key")
username.send_keys("mjeng@ucsc.edu")
password = browser.find_element_by_id("session_password")
password.send_keys("CSE115Project")

In [36]:
login_button=browser.find_element_by_class_name("sign-in-form__submit-button")
login_button.click()

In [37]:
browser.get("https://www.linkedin.com/jobs")

In [38]:
job_search_bar = browser.find_element_by_id("jobs-search-box-keyword-id-ember37")
job_search_bar.send_keys("Software Engineer")

In [41]:
job_search_button = browser.find_element_by_class_name("jobs-search-box__submit-button")
job_search_button.click()

In [46]:
#Get page source code
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')
#Strip text from source code
results = soup.find('small', {'class': 'display-flex t-12 t-black--light t-normal'}).get_text().strip().split()[0]
results = int(results.replace(',', ''))

TypeError: object of type 'NoneType' has no len()